# WAVPY TUTORIAL

This tutorial provides several examples for exploiting the capabilitites of *wavpy*. Different classes are employed to show some of their available functionalities to end up with a simulation of waveforms and DDM, which require the combination of the previous ones. 

In [ ]:
# List of imports: essentially wavpy and matplotlib to show results

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import sys
import os
HOME = os.getenv("HOME")
wavpyDIR = HOME + "/gnssr_analysis/trunk/src/waveform_pylib/wavpy/"
sys.path.insert(1,wavpyDIR)
import wavpy
import time

# 1: Composite GNSS signal

In [ ]:
mySignal = wavpy.GNSS_composite()

# We construct different GNSS signals by modifying the weights of the codes available

[range_signal, gps_all_codes] = mySignal.get_lambda_func(mySignal.lambda_size, mySignal.lambda_size)
plt.plot(range_signal, gps_all_codes, 'b-', label='GPS all codes')

mySignal.weight_E1A = 1.0
mySignal.weight_E1B = 1.0
mySignal.weight_E1C = 1.0
mySignal.weight_CA = 0.0
mySignal.weight_PY = 0.0
mySignal.weight_M = 0.0
mySignal.weight_IM = 0.0
mySignal.compute_lambda_func()
[range_signal, galileo_all_codes] = mySignal.get_lambda_func(mySignal.lambda_size, mySignal.lambda_size)
plt.plot(range_signal, galileo_all_codes, 'g-', label='Galileo all codes')

mySignal.weight_CA = 1.0
mySignal.weight_E1A = 0.0
mySignal.weight_E1B = 0.0
mySignal.weight_E1C = 0.0
mySignal.compute_lambda_func()
[range_signal, gps_L1CA] = mySignal.get_lambda_func(mySignal.lambda_size, mySignal.lambda_size)
plt.plot(range_signal, gps_L1CA, 'r-', label='GPS L1 C/A')

plt.xlabel("Range [m]")
plt.ylabel("Normalized power")
plt.grid()
plt.legend()
plt.show()

# 2: Reflecting surface

In [ ]:
mySurface = wavpy.Reflecting_surface()

# We first check permittivity aspects

# We built a function to plot the Fresnel reflection coefficients for circular polarizations
def plot_all_Rfresnel(wavpy_surf, labelname, colorline):
    incidence_angles = np.arange(0.0, 91.0, 1.0)
    epsilon_air = [1.0, 0.0]
    rco = np.zeros(len(incidence_angles))
    rcross = np.zeros(len(incidence_angles))
    for i in range(len(incidence_angles)):
        [[rco_r, rco_i], [rcross_r, rcross_i]] = wavpy_surf.compute_Rfresnel_circular(incidence_angles[i], epsilon_air)
        rco[i] = np.sqrt(rco_r*rco_r + rco_i*rco_i)
        rcross[i] = np.sqrt(rcross_r*rcross_r + rcross_i*rcross_i)
    plt.plot(incidence_angles, rco, '--', color=colorline)
    plt.plot(incidence_angles, rcross, '-', color=colorline, label=labelname)
    return

# Sea water as the default case
plot_all_Rfresnel(mySurface, 'Sea water', 'r')

# Case of new/young sea ice (direct setting of relative permittivity values)
mySurface.epsilon_real = 10.0/np.sqrt(2.0)
mySurface.epsilon_imag = 10.0/np.sqrt(2.0)
plot_all_Rfresnel(mySurface, 'Sea ice (New/Young)', 'm')

# Case of first/multi-year sea ice (computed from brine volume)
mySurface.epsilon_sea_ice(30.0)
plot_all_Rfresnel(mySurface, 'Sea ice (First/Multi-Year)', 'y')

# Case of wet snow (computed from snow density and water volume)
mySurface.epsilon_wet_snow(0.5, 8.0)
plot_all_Rfresnel(mySurface, 'Wet Snow', 'g')

# Case of dry snow (computed from snow density)
mySurface.epsilon_dry_snow(0.5)
plot_all_Rfresnel(mySurface, 'Dry Snow', 'b')

# Show the results obtained
plt.xlim([0.0, 90.0])
plt.xlabel("Incidence angle [deg]")
plt.ylim([0.0, 1.0])
plt.ylabel("Reflection coef.")
plt.grid()
plt.legend()
plt.show()

In [ ]:
# We focus now on roughness 

# We can directly set the MSS values of the surface or, in the case of sea water, we derive them using different methods:

# A) From wind speed using [Katzberg et al, 06]

mySurface.wind_U10_speed = 20.0
mySurface.wind_U10_azimuth = 0.0
mySurface.compute_mss_from_wind()

print("MSS values: [%f, %f]" % (mySurface.mss_x, mySurface.mss_y))

In [ ]:
# B) Constructing a sea spectrum based on [Elfouhaily et al, 97]

InvWavAge = 0.84
WindWavesAngle = 0.0
n = 4096
dk = 2.0*np.pi/410.0
# This generates a spectrum evaluated at n values in kx and n values in ky,
# from -n/2 ... to ... n/2
# Therefore, the minimum and maximum kx and ky are: -n/2*dk to +n/2*dk
mySurface.compute_sea_spectrum(n, dk, WindWavesAngle, InvWavAge)

# Then, we can compute the corresponding MSS values from the stored spectrum (which could also be inputed by the user)
mySurface.compute_mss_from_spectrum()

print("MSS values: [%f, %f]" % (mySurface.mss_x, mySurface.mss_y))

# 3: Specular geometry

In [ ]:
myGeom = wavpy.Specular_geometry()

def dump_geo_params(wavpy_geo):
    posT = wavpy_geo.get_ECEFpos_Transmitter()
    velT = wavpy_geo.get_ECEFvel_Transmitter()
    coorT = wavpy_geo.get_LongLatHeight_Transmitter()
    print("==================> Transmitter")
    print("- Pos-ECEF [km]: %f %f %f" % (posT[0], posT[1], posT[2]))
    print("- Vel-ECEF [km/s]: %f %f %f" % (velT[0], velT[1], velT[2]))
    print("- Coordinates [deg, deg, km]: %f %f %f" % (coorT[0], coorT[1], coorT[2]))
    print("- Azimuth, Elevation [deg]: %f %f" % (wavpy_geo.azimuthT, wavpy_geo.elevation))
    posR = wavpy_geo.get_ECEFpos_Receiver()
    velR = wavpy_geo.get_ECEFvel_Receiver()
    coorR = wavpy_geo.get_LongLatHeight_Receiver()
    print("==================> Receiver")
    print("- Pos-ECEF [km]: %f %f %f" % (posR[0], posR[1], posR[2]))
    print("- Vel-ECEF [km/s]: %f %f %f" % (velR[0], velR[1], velR[2]))
    print("- Coordinates [deg, deg, km]: %f %f %f" % (coorR[0], coorR[1], coorR[2]))
    print("- Azimuth, Elevation [deg]: %f %f" % (wavpy_geo.azimuthR, wavpy_geo.elevation))
    posS = wavpy_geo.get_ECEFpos_Specular()
    undu = wavpy_geo.get_Undulation()
    print("==================> Specular point")
    print("- Pos-ECEF [km]: %f %f %f" % (posS[0], posS[1], posS[2]))
    print("- Coordinates [deg, deg, km]: %f %f %f" % (wavpy_geo.longitudeS, wavpy_geo.latitudeS, undu))
    return

# There are different options to set a bistatic-radar scenario:

# A) Setting ECEF positions of receiver and transmiter

myGeom.set_ECEFpos_Receiver([3061.222, -5984.822, 1544.057])
myGeom.set_ECEFpos_Transmitter([4080.0950610, -15319.6341923, 21324.6634863])
myGeom.compute_specular_point(1)
dump_geo_params(myGeom)

In [ ]:
# B) Setting positions from Lon-Lat-Height coordinates or interpolating them from ascii files

myGeom.set_LongLatHeight_Receiver([-62.9103656, 13.0138428, 520.2783949])
myGeom.read_ECEFpos_GNSS_Transmitter("data/igs19672.sp3", 1967, 230576, 6, 'G')
myGeom.compute_specular_point(1)
dump_geo_params(myGeom)

In [ ]:
# C) Simply by chosing a specular point location and the relative orientation of receiver and transmitter wrt it

lonSpecular = 14.782621
latSpecular = 41.129761
heightS = 0.0
elevation = 60.0
heightR = 520.0
heightT = 20000.0
azimT = 90.0
myGeom.set_geometry_from_ElevHeightsSpec(elevation, heightR, heightT, lonSpecular, latSpecular, azimT, heightS, 0)
myGeom.set_tangEarthVel_Receiver(8.0, 0.0)
myGeom.set_tangEarthVel_Transmitter(3.0, 0.0)
dump_geo_params(myGeom)

# 4: Receiver Front-End

In [ ]:
myReceiver = wavpy.RF_FrontEnd()

# We define the antenna pattern from a set of points in both E and H-planes 
anglesE = [-90.0, -53.0, -40.0, -25.0, 0.0, 20.0, 35.0, 45.0, 90.0]
patternE = [-40.0, -20.0, -11.0, -4.0, 0.0, -3.0, -9.0, -15.0, -40.0]
anglesH = [-60.0, -52.0, -45.0, -37.0, -30.0, -28.0, -25.0, -20.0, -15.0, 0.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 45.0, 60.0]
patternH = [-40.0, -20.0, -15.0, -13.0, -16.0, -20.0, -40.0, -14.0, -6.0, 0.0, -3.0, -8.0, -15.0, -40.0, -18.0, -15.7, -20.0, -40.0]

ant_gain = 15.0
ant_temp = 200.0
noise_F = 3.0
baseband_BW = 5000000.0
isotropic_ant = 0
myReceiver.set_receiver_params(ant_gain, ant_temp, noise_F, baseband_BW, isotropic_ant)
myReceiver.set_antenna_patterns_interp(anglesE, patternE, anglesH, patternH, -40.0)
myReceiver.set_antenna_orientation_BF_EH([0.0, 1.0, 0.0], [-1.0, 0.0, 0.0])

# Plot antenna diagram
[pattern_E_out, pattern_H_out] = myReceiver.get_antenna_patterns()
angles_out = np.concatenate((np.arange(0,181,1), np.arange(-179,0,1)), axis=0)
plt.plot(angles_out, pattern_E_out - pattern_E_out[0], label=r'E-plane ($\phi =$  0|180 deg)')
plt.plot(angles_out, pattern_H_out - pattern_H_out[0], label=r'H-plane ($\phi =$ 90|270 deg)')
plt.plot(anglesE, patternE, '.')
plt.plot(anglesH, patternH, '.')
plt.xlabel(r'$\theta$ [deg]')
plt.ylabel('Gain [dB]')
plt.grid()
plt.legend(loc=1)
plt.xlim([-179, 180.0])
plt.ylim([-40.0, 5.0])
plt.show()

In [ ]:
# We define a function to plot 3D representations of the radiation pattern

def plot_ant_pattern(wavpy_rec, apply_AF):
    ant_gain = wavpy_rec.get_antenna_whole_pattern()
    if(apply_AF):
        ant_gain = ant_gain + np.nan_to_num(wavpy_rec.get_array_factor())
        ant_gain = np.clip(ant_gain, 0.0, None)       
    phi, theta = np.meshgrid(np.arange(0,360), np.arange(0,181))
    fig1 = plt.figure(1)
    ax1 = plt.axes(projection='3d')
    ax1.plot_surface(theta, phi, ant_gain, cmap=plt.cm.jet, rstride=1, cstride=1, linewidth=0)
    ax1.set_xlabel(r'$\theta$ [deg]')
    ax1.set_ylabel(r'$\phi$ [deg]')
    ax1.set_zlabel('Gain [dB]')
    ax1.set_title('Antenna pattern')
    min_ant_gain = np.min(np.min(ant_gain[:,:]))
    x = (ant_gain - min_ant_gain)*np.sin(theta*np.pi/180.0)*np.cos(phi*np.pi/180.0)
    y = (ant_gain - min_ant_gain)*np.sin(theta*np.pi/180.0)*np.sin(phi*np.pi/180.0)
    z = (ant_gain - min_ant_gain)*np.cos(theta*np.pi/180.0)
    fig2 = plt.figure(2)
    ax2 = plt.axes(projection='3d')
    ax2.plot_surface(x, y, z, cmap=plt.cm.jet, rstride=1, cstride=1, linewidth=0)
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Z')
    ax2.set_title('Radiation pattern')
    fig3 = plt.figure(3)
    ax3 = plt.axes(projection='3d')
    ax3.view_init(elev=0.0, azim=0.0)
    ax3.plot_surface(x, y, z, cmap=plt.cm.jet, rstride=1, cstride=1, linewidth=0)
    ax3.set_xlabel('X')
    ax3.set_zlabel('Z')
    ax3.set_title('Radiation pattern')
    fig4 = plt.figure(4)
    ax4 = plt.axes(projection='3d')
    ax4.view_init(elev=0.0, azim=270.0)
    ax4.plot_surface(x, y, z, cmap=plt.cm.jet, rstride=1, cstride=1, linewidth=0)
    ax4.set_ylabel('Y')
    ax4.set_zlabel('Z')
    ax4.set_title('Radiation pattern')
    plt.show()

# Plot the previously loaded antenna pattern 
print("Plotting antenna patterns...")
plot_ant_pattern(myReceiver, False)

In [ ]:
# We set a new anttena pattern from an ascii file and plot the results

ant_gain_array = np.loadtxt("data/cygnss_antpattern_deg.txt", unpack=True)
ant_pattern_CYGNSS = np.zeros([181,360]) - 10.0
for i in range(91):
    for j in range(360):
        ant_pattern_CYGNSS[i, j] = ant_gain_array[i*360 + j]
        
myReceiver.set_receiver_params(0.0, ant_temp, noise_F, baseband_BW, isotropic_ant)
myReceiver.set_antenna_whole_pattern(ant_pattern_CYGNSS)

print("Plotting antenna patterns...")
plot_ant_pattern(myReceiver, False)

In [ ]:
# Set a uniform planar array as antenna

num_elem_1D = 16
theta_max = 45.0
phi_max = 270.0

# We first load the antenna pattern from a single element (basic hemispherial antenna)
angles = [-120.0, -90.0, -50.0, 0.0, 50.0, 90.0, 120.0]
pattern = [-20.0, -5.0, -0.5, 0.0, -0.5, -5.0, -20.0]
myReceiver.set_receiver_params(3.0, ant_temp, noise_F, baseband_BW, isotropic_ant)
myReceiver.set_antenna_pattern_interp(angles, pattern, -20.0)

# We define the location of the elements in the array (a square equi-spaced one with half lambda separations)
pos_elems = np.zeros([num_elem_1D*num_elem_1D,2])
for i in range(num_elem_1D):
    for j in range(num_elem_1D):
        pos_elems[i*num_elem_1D + j][0] = 0.5*i - (0.5*(num_elem_1D - 1))/2.0
        pos_elems[i*num_elem_1D + j][1] = 0.5*j - (0.5*(num_elem_1D - 1))/2.0         
myReceiver.set_antenna_elements_pos_AF(pos_elems, 1)

# Configure the pointing direction of the array factor
myReceiver.compute_phase_delays_UPA(theta_max, phi_max)
myReceiver.compute_array_factor()

print("Plotting antenna patterns...")
plot_ant_pattern(myReceiver, True)

# 5: WAVEFORM AND DDM MODEL

Class *ZaVoModel_GNSSR* provides the functions to simulate waveforms and DDM's. It contains objects from the previous classes in order to charactize the different aspects of a GNSS-R scenario. Therefore, the user may edit and check the contents of each element before running a simulation. The next code lines provide different exmaples of use.

In [ ]:
myDDMmodel = wavpy.ZaVoModel_GNSSR()

myDDMmodel.sampling_rate = 4091750.0
myDDMmodel.wav_length = 64
myDDMmodel.delta_doppler = 500.0
myDDMmodel.ddm_half_dopplers = 10
# Previous GNSS signal: GPS L1 C/A
myDDMmodel.gnss_signal = mySignal
# Sea water surface with a roughness level computed from wind speed
myDDMmodel.surface.wind_U10_speed = 20.0
myDDMmodel.surface.compute_mss_from_wind()
# A real CYGNSS case
myDDMmodel.geometry.set_ECEFpos_Receiver([3061.222, -5984.822, 1544.057])
myDDMmodel.geometry.set_ECEFvel_Receiver([4.831, 3.526, 4.01])
myDDMmodel.geometry.set_inertials(0.0, 0.0, 55.66)
myDDMmodel.geometry.read_ECEFpos_GNSS_Transmitter("data/igs19672.sp3", 1967, 230576, 6, 'G')
myDDMmodel.geometry.compute_specular_point(1)
# We set the previous receiver characteristics using the loaded antenna pattern
myDDMmodel.receiver_Down.set_receiver_params(0.0, ant_temp, noise_F, baseband_BW, isotropic_ant)
myDDMmodel.receiver_Down.set_antenna_whole_pattern(ant_pattern_CYGNSS)

# Compute waveform model and DDM based on [Zavorotny & Voronovich, 00]

start_time = time.time()
#myDDMmodel.compute_waveform(0, 1, 0, 0, 0)
myDDMmodel.interferometric_flag = False
myDDMmodel.curvature_approx_flag = True
myDDMmodel.covariance_wav_flag = False
myDDMmodel.covariance_ddm_flag = False
myDDMmodel.recompute_Lambda_flag = True
myDDMmodel.compute_waveform()
print("=> Computation Time for waveform and DDM: %s seconds" % (time.time() - start_time))

In [ ]:
# Plot PNR (peak to noise ratio) waveform

range_wav = myDDMmodel.waveform_POW.get_range_waveform(myDDMmodel.wav_length)
range_spec = range_wav - myDDMmodel.geometry.geometric_delay*1000.0
wav_pow = myDDMmodel.waveform_POW.get_waveform(myDDMmodel.wav_length)
noise_level = wav_pow[0]

plt.figure(1)
plt.plot(range_spec/1000.0, 10.0*np.log10(wav_pow/noise_level), '.-')
plt.grid()
plt.title("Example of CYGNSS waveform model (elevation = %3.2f deg)" % (myDDMmodel.geometry.elevation))
plt.xlabel("Range from specular [km]")
plt.ylabel("PNR [dB]")

# Plot DDM

DDM = np.zeros([(myDDMmodel.ddm_half_dopplers*2 + 1), myDDMmodel.wav_length])
for freq_ind in range(0, (myDDMmodel.ddm_half_dopplers*2 + 1)):
    DDM[freq_ind, :] = myDDMmodel.get_DDM_doppler_slice((myDDMmodel.ddm_half_dopplers - freq_ind), myDDMmodel.wav_length)

plt.figure(2)
plt.imshow(DDM[:,:], extent=[range_spec[0]/1000.0, range_spec[-1]/1000.0, -myDDMmodel.delta_doppler*float(myDDMmodel.ddm_half_dopplers), myDDMmodel.delta_doppler*float(myDDMmodel.ddm_half_dopplers)], vmin=noise_level, vmax=max(DDM[myDDMmodel.ddm_half_dopplers,:]), cmap='jet', aspect='auto')
plt.title("Example of CYGNSS DDM model (elevation = %3.2f deg)" % (myDDMmodel.geometry.elevation))
plt.xlabel("Range from specular [km]")
plt.ylabel("Doppler [Hz]")
plt.colorbar()

plt.show()

In [ ]:
# Compute waveform covariance model based on [Li et al, 17]

start_time = time.time()
#myDDMmodel.compute_waveform(0, 1, 1, 1, 0)
myDDMmodel.covariance_wav_flag = True
myDDMmodel.compute_waveform()
print("=> Computation Time for waveform: %s seconds" % (time.time() - start_time))

In [ ]:
# Plot PNR (peak to noise ratio) waveform

range_wav = myDDMmodel.waveform_POW.get_range_waveform(myDDMmodel.wav_length)
range_spec = range_wav - myDDMmodel.geometry.geometric_delay*1000.0
wav_mean = myDDMmodel.waveform_POW.get_waveform(myDDMmodel.wav_length)

plt.figure(1)
plt.plot(range_spec/1000.0, 10.0*np.log10(wav_pow/noise_level), 'b.-', label="Standard method")
plt.plot(range_spec/1000.0, 10.0*np.log10(wav_mean/wav_mean[0]), 'r.-', label="Covariance method")
plt.grid()
plt.xlabel("Range from specular [km]")
plt.ylabel("PNR [dB]")
plt.legend(loc=1)

# Plot waveform covariance model

cov_wav = np.zeros([myDDMmodel.wav_length, myDDMmodel.wav_length])
for cov_ind in range(0, myDDMmodel.wav_length):
    cov_wav[myDDMmodel.wav_length - 1 - cov_ind, :] = myDDMmodel.get_cov_slice(cov_ind, myDDMmodel.wav_length)

plt.figure(2)
plt.imshow(cov_wav[:,:], extent=[range_spec[0]/1000.0, range_spec[-1]/1000.0, range_spec[0]/1000.0, range_spec[-1]/1000.0], vmin=min(cov_wav.min(axis=0)), vmax=max(cov_wav.max(axis=0)), cmap='jet', aspect='auto')
plt.title("Waveform covariance model")
plt.xlabel("Range from specular [km]")
plt.ylabel("Range from specular [km]")
plt.colorbar()

plt.show()

In [ ]:
# Computing noisy waveforms using stored covariance model and mean waveform

plt.figure(1)
num_wavs = 1000
noisy_wavs = np.zeros([num_wavs, myDDMmodel.wav_length])
start_time = time.time()
for i in range(num_wavs):
    seed_random_generator = i
    noisy_wavs[i,:] = myDDMmodel.get_noisy_waveform(myDDMmodel.wav_length, i)
    if(i < 10):# Plot the 10 first noisy waveforms
        plt.plot(range_spec/1000.0, 10.0*np.log10(noisy_wavs[i,:]/wav_mean[0]), '--')
print("=> Computation Time for 1000 noisy waveform: %s seconds" % (time.time() - start_time))

plt.plot(range_spec/1000.0, 10.0*np.log10(wav_mean/wav_mean[0]), 'k.-', label="Mean waveform")
plt.grid()
plt.title("Example of 10 simulated noisy waveforms")
plt.xlabel("Range from specular [km]")
plt.ylabel("PNR [dB]")
plt.legend(loc=1)

# Plot covariance from 1000 simulated waveforms

plt.figure(2)
cov_sim = np.cov(noisy_wavs, rowvar=False)
plt.imshow(np.flipud(cov_sim[:,:]), extent=[range_spec[0]/1000.0, range_spec[-1]/1000.0, range_spec[0]/1000.0, range_spec[-1]/1000.0], vmin=min(cov_sim.min(axis=0)), vmax=max(cov_sim.max(axis=0)), cmap='jet', aspect='auto')
plt.title("Covariance from 1000 simulated noisy wavs.")
plt.xlabel("Range from specular [km]")
plt.ylabel("Range from specular [km]")
plt.colorbar()

plt.show()

In [ ]:
# Compute DDM covariance model based on [Li et al, 17]

start_time = time.time()
#myDDMmodel.compute_waveform(0, 1, 1, 1, 1)
myDDMmodel.covariance_ddm_flag = True
myDDMmodel.compute_waveform()
print("=> Computation Time for waveform and DDM: %s seconds" % (time.time() - start_time))

# Plot DDM
DDM_mean = np.zeros([(myDDMmodel.ddm_half_dopplers*2 + 1), myDDMmodel.wav_length])
for freq_ind in range(0, (myDDMmodel.ddm_half_dopplers*2 + 1)):
      DDM_mean[freq_ind, :] = myDDMmodel.get_DDM_doppler_slice((myDDMmodel.ddm_half_dopplers - freq_ind), myDDMmodel.wav_length)

plt.imshow(DDM_mean[:,:], extent=[range_spec[0]/1000.0, range_spec[-1]/1000.0, -myDDMmodel.delta_doppler*float(myDDMmodel.ddm_half_dopplers), myDDMmodel.delta_doppler*float(myDDMmodel.ddm_half_dopplers)], vmin=min(DDM_mean[myDDMmodel.ddm_half_dopplers,:]), vmax=max(DDM_mean[myDDMmodel.ddm_half_dopplers,:]), cmap='jet', aspect='auto')
plt.title("Mean DDM from covariance model")
plt.xlabel("Range from specular [km]")
plt.ylabel("Doppler [Hz]")
plt.colorbar()

plt.show()

In [ ]:
# Compute and plot 1 noisy DDM from covariance model

seed_random_generator = 1
start_time = time.time()
DDM_noise = myDDMmodel.get_noisy_DDM(myDDMmodel.wav_length*(myDDMmodel.ddm_half_dopplers*2 + 1), seed_random_generator)
print("=> Computation Time for 1 noisy DDM: %s seconds" % (time.time() - start_time))

# Plot noisy DDM
DDM_noise_plot = np.flipud(DDM_noise.reshape((myDDMmodel.ddm_half_dopplers*2 + 1), myDDMmodel.wav_length))

plt.imshow(DDM_noise_plot[:,:], extent=[range_spec[0]/1000.0, range_spec[-1]/1000.0, -myDDMmodel.delta_doppler*float(myDDMmodel.ddm_half_dopplers), myDDMmodel.delta_doppler*float(myDDMmodel.ddm_half_dopplers)], vmin=min(DDM_mean[myDDMmodel.ddm_half_dopplers,:]), vmax=max(DDM_mean[myDDMmodel.ddm_half_dopplers,:]), cmap='jet', aspect='auto')
plt.title("Example of noisy DDM from covariance model")
plt.xlabel("Range from specular [km]")
plt.ylabel("Doppler [Hz]")
plt.colorbar()

plt.show()